In [1]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


2025-06-05 11:44:42.224469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 11:44:42.388821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749120282.454217   77537 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749120282.472489   77537 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749120282.615736   77537 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
metadata = pd.read_csv("HAM10000_metadata.csv")

metadata

FileNotFoundError: [Errno 2] No such file or directory: 'HAM10000_metadata.csv'

In [ ]:
# Step 2: Load Metadata
metadata = pd.read_csv("HAM10000_metadata.csv")
label_map = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
metadata['label'] = metadata['dx'].map(label_map)
metadata

In [ ]:
# Step 3: Load and Resize Images
image_dir = "HAM10000_all_images"  # Adjust if using part_2 or combined folder
IMG_SIZE = 64

images = []
labels = []

for i, row in metadata.iterrows():
    img_path = os.path.join(image_dir, row['image_id'] + '.jpg')
    if os.path.exists(img_path):
        img = Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))
        images.append(np.array(img) / 255.0)
        labels.append(row['label'])

X = np.array(images)
y = to_categorical(labels, num_classes=7)

print("Image shape:", X.shape)
print("Label shape:", y.shape)

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np

# Load metadata
metadata = pd.read_csv("HAM10000_metadata.csv")
label_map = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
metadata['label'] = metadata['dx'].map(label_map)

# ✅ Adjust this if needed
image_dir = "HAM10000_all_images"  # or "HAM10000_images_part_1" if you only have part 1

# Load images
images = []
labels = []
missing = 0

for i, row in metadata.iterrows():
    img_file = row['image_id'] + ".jpg"
    img_path = os.path.join(image_dir, img_file)
    if os.path.exists(img_path):
        try:
            img = Image.open(img_path).resize((64, 64))
            images.append(np.array(img) / 255.0)
            labels.append(label_map[row['dx']])
        except:
            missing += 1
    else:
        missing += 1

print("✅ Loaded images:", len(images))
print("❌ Missing or failed images:", missing)


In [ ]:
# Step 4: Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Step 5: Define the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # 7 classes
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# Step 6: Train the Model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=32)


In [ ]:
# Step 7: Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('CNN Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("ham10000_cnn_model.h5")

# Load your iPhone image
your_img = Image.open("your_image.jpg").resize((64, 64))
your_img_array = np.expand_dims(np.array(your_img) / 255.0, axis=0)

prediction = model.predict(your_img_array)
classes = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print("Predicted Lesion Type:", classes[np.argmax(prediction)])

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# Load metadata
metadata = pd.read_csv("HAM10000_metadata.csv")

# Choose the correct image folder
image_dirs = ["HAM10000_all_images"]  # combine both parts


In [ ]:

n_images = 25
img_size = (64, 64)

# Create a plot
plt.figure(figsize=(15, 15))


count = 0
for i, row in metadata.iterrows():
    if count >= n_images:
        break

    img_file = row['image_id'] + ".jpg"
    label = row['dx']


    for d in image_dirs:
        img_path = os.path.join(d, img_file)
        if os.path.exists(img_path):
            img = Image.open(img_path).resize(img_size)
            ax = plt.subplot(5, 5, count + 1)
            plt.imshow(img)
            plt.title(label)
            plt.axis("off")
            count += 1
            break  # found image, stop searching other folders

plt.tight_layout()
plt.show()


In [ ]:
metadata['dx'].unique()


In [ ]:
metadata['dx'].value_counts()
